In [1]:
import os
import logging
from os.path import abspath
from pathlib import Path
from dataclasses import field
from typing import Optional
from datasets import load_dataset
import transformers
import torch
import numpy as np
from transformers import (
    T5Tokenizer,
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    TrainingArguments,
)
from wasabi import msg

/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


# Load Config

In [2]:
# Load the config
import yaml
with open('config/config_testing.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
# Pretty print the config using json as a shortcut
import json
print(json.dumps(config, sort_keys=False, indent=4))

{
    "model_name": "google/t5-v1_1-base",
    "dataset_vars": {
        "type": "csv",
        "dir": "data/cdr_seq2rel",
        "split": "train",
        "column_names": [
            "input",
            "relations"
        ]
    },
    "output_dir": "./fine_tune_results",
    "local_rank": -1,
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 8,
    "learning_rate": 0.0002,
    "max_grad_norm": 0.3,
    "weight_decay": 0.001,
    "max_seq_length": 512,
    "max_target_length": 512,
    "num_train_epochs": 0.5,
    "packing": false,
    "padding": true,
    "ignore_pad_token_for_loss": true,
    "truncation": true,
    "predict_with_generate": true,
    "gradient_checkpointing": true,
    "optim": "paged_adamw_32bit",
    "lr_scheduler_type": "constant",
    "max_steps": 12,
    "warmup_ratio": 0.03,
    "group_by_length": true,
    "save_steps": 4,
    "do_eval": true,
    "evaluation_strategy": "steps",
    "eval_steps":

# Load dataset

First we'll load the dataset. The dataset used is the [CDR dataset in seq2rel format](https://github.com/JohnGiorgi/seq2rel-ds), This dataset is slightly altered to make it easier to use with [huggingface dataset](https://huggingface.co/docs/datasets/index).

In [4]:
dataset = load_dataset(
        config['dataset_vars']['type'], 
        data_dir=config['dataset_vars']['dir'],
        column_names=config['dataset_vars']['column_names']
        )

dataset_train = dataset['train'].select(range(1,501)) # remove first row that contains column names
dataset_eval = dataset['validation'].select(range(1,501)) # remove first row that contains column names

In [5]:
# Useful function to see a random row in the dataset
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset_train, 1)

,input,relations
0,"Central nervous system complications during treatment of acute lymphoblastic leukemia in a single pediatric institution. Central nervous system (CNS) complications during treatment of childhood acute lymphoblastic leukemia (ALL) remain a challenging clinical problem. Outcome improvement with more intensive chemotherapy has significantly increased the incidence and severity of adverse events. This study analyzed the incidence of neurological complications during ALL treatment in a single pediatric institution, focusing on clinical, radiological, and electrophysiological findings. Exclusion criteria included CNS leukemic infiltration at diagnosis, therapy-related peripheral neuropathy, late-onset encephalopathy, or long-term neurocognitive defects. During a 9-year period, we retrospectively collected 27 neurological events (11%) in as many patients, from 253 children enrolled in the ALL front-line protocol. CNS complications included posterior reversible leukoencephalopathy syndrome (n = 10), stroke (n = 5), temporal lobe epilepsy (n = 2), high-dose methotrexate toxicity (n = 2), syndrome of inappropriate antidiuretic hormone secretion (n = 1), and other unclassified events (n = 7). In conclusion, CNS complications are frequent events during ALL therapy, and require rapid detection and prompt treatment to limit permanent damage.",methotrexate @CHEMICAL@ central nervous system complications ; central nervous system (cns) complications ; neurological complications ; neurocognitive defects @DISEASE@ @CID@


The dataset has 2 columns, "Input" which are abstracts from various papers from pubmed and "relations" which are the relations between chemicals and diseases. The relations are in an novel format explained in the [seq2rel paper](https://aclanthology.org/2022.bionlp-1.2/). 

# Load tokenizer and model

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [8]:
model_name = config['model_name']
device_map = {"": 0}

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    device_map=device_map
) # we specificly use T5 for COnfitional generations because it has a language modeling head

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


KeyboardInterrupt: 

# Test tokenizer and model

In [ ]:
# Show a single example of the dataset
single_example = dataset_train[0]
print(f"A single row in the dataset is of type {type(single_example)}")
print(single_example)

Before going into the model the data has to be tokenized. Tokenization is a preprocess where the text is split up into tokens, these tokens are then conferted to their respective token ids using the vocab, token ids can then be converted into word vectors which are the raw input of the model. The attention mask shows which tokens should be given attention to in the attention layer in the model. A 1 means the token is given attention to in the attention layer a 0 means the token is ignored in that process. This is important when using padding for example, you don't want the padding tokens to be factored in during the attention process. 

In [ ]:
column_name_input, column_name_output = config['dataset_vars']['column_names']

In [ ]:
# Show what the output of the tokenizer looks like for the input task
tokenizer_output_text = tokenizer(single_example[column_name_input])
tokenizer_output_text

In [ ]:
# Show what the output of the tokenizer looks like for the output text
with tokenizer.as_target_tokenizer():
    tokenizer_output_rel = tokenizer(single_example[column_name_output])
tokenizer_output_rel

Why tokenize the output? During training the model tasks is given a certain input match the expected output. To match it it needs to know where it's going wrong, so for the calculation of loss it needs the input ids of the tokenized output text. The exact training algorithm is presuambly [teacher forcing](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/). (I say presuamly here because i don't really know what training algoritm the huggingface api uses, where it's documentated, defined in the code and if it differs between models. And teacher forcing was used for training the T5 model)

The T5 model has a max sequence input of 512. If an input exceed that it is truncated. This could cause the model to output the wrong relations because it is missing some information, certainly when a lot of important information output the conclusion of a paper is usally at the end of an abstract. So we'll check how many rows exceed the max sequence length:

In [ ]:
tokenized_dataset = tokenizer(dataset_train[column_name_input])
count = 0
for idx, input_ids in enumerate(tokenized_dataset['input_ids']):
    if len(input_ids) > 512:
        # print(idx, len(input_ids))
        count+=1
print(f'Percentage that comes above max sequence length: {count/(idx+1)*100}')

# Create Preprocessing function

In [ ]:
ignore_pad_token_for_loss=config['ignore_pad_token_for_loss']

In [ ]:
def preprocess_function(examples):
    '''
    This function takes a dataset of input and target sequences.
    meant to be used with the dataset.map() function
    '''

    column_name_input, column_name_output = config['dataset_vars']['column_names']

    # Split input and target
    inputs, targets = [], []
    for i in range(len(examples[column_name_input])):
        if examples[column_name_input][i] and examples[column_name_output][i]: # remove pairs where one is None
            inputs.append(examples[column_name_input][i])
            targets.append(examples[column_name_output][i])

    # Tokenize the input
    model_inputs = tokenizer(
        inputs, 
        max_length=config['max_seq_length'], 
        padding=config['padding'], 
        truncation=config['truncation'], 
        return_tensors='pt'
    )

    # Tokenize the target sequence
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, 
            max_length=config['max_seq_length'], 
            padding=config['padding'], 
            truncation=config['truncation'], 
            return_tensors='pt'
        )

    # Replace pad tokens with -100 so they don't contribute too the loss
    if ignore_pad_token_for_loss:
        labels["input_ids"] = [
                    [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
                ]

    # Add tokenized target text to output
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
train_dataset = dataset_train.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset"
)

eval_dataset = dataset_eval.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset"
)

In [ ]:
# A look into the pre-tokenized dataset
single_example = train_dataset[0]
print(f"A single row in the dataset is of type {type(single_example)}")
print(single_example)

# Create evaluation strategy

In [ ]:
#model.generate(single_example)

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load("rouge")

In [ ]:
single_eval_example=[dataset_eval[0]['relations']]
print(single_eval_example)
# whats the output of metric.compute?
metric.compute(
    predictions=single_eval_example,
    references=single_eval_example,
    rouge_types=['rouge1', 'rouge2']
)


In [ ]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
metric.compute(predictions=predictions, references=references, rouge_types=['rouge1', 'rouge2'])

We need to define two functions:
- a postprocess function that does the post processing of a prediction
- A compute_metrics function that takes as input an prediction and output the mean score of that prediction?

the compute metrics functions alwas takes a tuple of 

In [ ]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        # preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        # labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    # Replace -100s used for padding as we can't decode them
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)
    result = {k: round(v * 100, 4) for k, v in result.items()} # rounds all metric values to 4 numvers behind the comma
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens) # mean length of the generated sequences
    return result

# Setting up trainer

In [ ]:
training_arguments = Seq2SeqTrainingArguments(
        output_dir=config['output_dir'],
        per_device_train_batch_size=config['per_device_train_batch_size'],
        gradient_accumulation_steps=config['gradient_accumulation_steps'],
        optim=config['optim'],
        save_steps=config['save_steps'],
        logging_steps=config['logging_steps'],
        learning_rate=config['learning_rate'],
        fp16=config['fp16'],
        bf16=config['bf16'],
        max_grad_norm=config['max_grad_norm'],
        max_steps=config['max_steps'],
        warmup_ratio=config['warmup_ratio'],
        group_by_length=config['group_by_length'],
        lr_scheduler_type=config['lr_scheduler_type'],
        predict_with_generate=True,
        save_total_limit=2,
        save_strategy='steps',
        load_best_model_at_end=True,
        do_eval=config['do_eval'],
        evaluation_strategy=config['evaluation_strategy'],
        eval_steps=config['eval_steps']
    )

In [ ]:
data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=-100,
        pad_to_multiple_of=8 if config['fp16'] else None,
    )

In [ ]:
trainer = Seq2SeqTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        args=training_arguments,
    )

# Training

In [ ]:
trainer.train()